In [1]:
import numpy as np
from scipy import integrate
import pandas as pd
import time

# 本项目目标为用不同数值方法计算积分并比较其差异
## 我们首先实现各种数值积分算法
### 复化梯形公式

In [2]:
def compound_trapezoidal(f, n, a, b):
    #f为函数，n + 1为求积节点数，[a,b]为积分区间
    h = (b - a) / n
    T1 = f(a) + f(b)
    T2 = np.matmul(np.ones(n - 1), f(a + h * np.arange(1, n)))
    T = h * (T1 + 2 * T2) / 2
    return T

### 复化Simpson公式

In [3]:
def compound_simpson(f, n, a, b):
    if n % 2 != 0:
        return "Error!"
    h = (b - a) / n
    T1 = f(a) + f(b)
    T2 = np.matmul(np.ones(int(n / 2) - 1), f(a + h * np.arange(2, n, 2)))
    T3 = np.matmul(np.ones(int(n / 2)), f(a + h * np.arange(1, n, 2)))
    T = h * (T1 + 2 * T2 + 4 * T3) / 3
    return T

### Romberg方法

In [4]:
def romberg():
    return

### Gauss型求积公式

In [5]:
def gauss_3(f, a, b):
    standard_gauss_point = np.array([-0.7745966692, 0, 0.7745966692])
    standard_A = np.array([0.5555555556, 0.8888888889, 0.5555555556])
    gauss_point = (b - a) * (standard_gauss_point + 1) / 2 + a
    A = (b - a) * standard_A / 2
    T = np.matmul(A, f(gauss_point))
    return T

def gauss_4(f, a, b):
    standard_gauss_point = np.array([-0.8611363116, -0.3399810436, 0.3399810436, 0.8611363116])
    standard_A = np.array([0.3478548451, 0.6521451549, 0.6521451549, 0.3478548451])
    gauss_point = (b - a) * (standard_gauss_point + 1) / 2 + a
    A = (b - a) * standard_A / 2
    T = np.matmul(A, f(gauss_point))
    return T

## 定义被积函数

In [6]:
def f(x):
    if not hasattr(x, '__iter__'):
        if x == 0:
            return 1
        return np.sin(x) / x
    return np.where(x != 0, np.sin(x) / x, 1)

## 计算数值积分

In [7]:
truth_value = integrate.quad(f, 0, np.pi / 2)[0]
print("真实值", truth_value)

result = pd.DataFrame([compound_trapezoidal(f, 8, 0, np.pi / 2),
                       compound_simpson(f, 8, 0, np.pi / 2),
                       gauss_3(f, 0, np.pi / 2),
                       gauss_4(f, 0, np.pi / 2)], index = ["复化梯形", "复化Simpson", "3节点Gauss型求积", "4节点Gauss型求积"], columns=["估计值"])

print(result)

真实值 1.3707621681544881
                  估计值
复化梯形         1.369460
复化Simpson    1.370764
3节点Gauss型求积  1.370763
4节点Gauss型求积  1.370762


## 计算误差等

In [8]:
err = np.ones_like(result["估计值"]) * truth_value - result["估计值"]
result.insert(loc=len(result.columns), column='误差值', value=err)

t = []
Stime = time.time()
for i in range(10000):
    compound_trapezoidal(f, 8, 0, np.pi / 2)
Etime = time.time()
t.append(Etime - Stime)

Stime = time.time()
for i in range(10000):
    compound_simpson(f, 8, 0, np.pi / 2)
Etime = time.time()
t.append(Etime - Stime)

Stime = time.time()
for i in range(10000):
    gauss_3(f, 0, np.pi / 2)
Etime = time.time()
t.append(Etime - Stime)

Stime = time.time()
for i in range(10000):
    gauss_4(f, 0, np.pi / 2)
Etime = time.time()
t.append(Etime - Stime)
t = np.array(t)

result.insert(loc = len(result.columns), column="计算时间*10000", value = t)
print(result)

                  估计值           误差值  计算时间*10000
复化梯形         1.369460  1.302559e-03    0.190490
复化Simpson    1.370764 -1.907924e-06    0.372005
3节点Gauss型求积  1.370763 -1.269137e-06    0.188496
4节点Gauss型求积  1.370762  2.723778e-09    0.194480
